In [1]:
import numpy as np
from scipy.spatial import distance_matrix
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def solve_tsp(locations):
    num_locations = len(locations)
    
    # Calculate distance matrix between locations
    dist_matrix = distance_matrix(locations, locations)
    
    # Create the routing index manager
    manager = pywrapcp.RoutingIndexManager(num_locations, 1, 0)
    
    # Create the routing model
    routing = pywrapcp.RoutingModel(manager)
    
    # Define distance callback (returns the distance between two locations)
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return dist_matrix[from_node][to_node]
    
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    # Set the search parameters
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    # Solve the problem
    solution = routing.SolveWithParameters(search_parameters)
    
    # Extract the solution
    route = []
    index = routing.Start(0)
    while not routing.IsEnd(index):
        node = manager.IndexToNode(index)
        route.append(locations[node])
        index = solution.Value(routing.NextVar(index))
    route.append(locations[manager.IndexToNode(index)])
    
    return route


# Example usage
locations = np.array([
    [0, 0],  # Starting point
    [1, 3],  # Delivery location 1
    [5, 2],  # Delivery location 2
    [3, 6],  # Delivery location 3
    [2, 4],  # Delivery location 4
])

# Solve the TSP to find the optimal route
optimal_route = solve_tsp(locations)

# Print the optimal route
for i, location in enumerate(optimal_route):
    print(f"Stop {i+1}: {location}")


Stop 1: [0 0]
Stop 2: [2 4]
Stop 3: [3 6]
Stop 4: [5 2]
Stop 5: [1 3]
Stop 6: [0 0]
